# Monthly vs. Hourly Mismatch Evaluation, all results in 1 CSV file

## A) Fixed Mismatch

The below example creates a scene and runs the timestamps between the two dates specified with and without torquetube in the module definition. It saves all results into two big excel files (one for torquetube on and one for torquetube off).



In [1]:
try:
    from bifacial_radiance import *
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.
    
import numpy as np
import os
import csv

In [2]:
# Simulation Control Parameters
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Demo4'
timeindex = 4020 # Noon, June 17th. 
simulationname = 'New_Panel'
timeindexstart=2150
timeindexend=2152

In [3]:
# MakeModule Parameters
numpanels = 1
x = 1  
y = 2
xgap = 0.01
ygap = 0.0
zgap = 0.02
sensorsy = 200 

# TorqueTube Parameters
axisofrotationTorqueTube=False # Doesn't matter, this is a fixed tilt simulation
torqueTube = True  # Gets re-defined in the foo loop.
tubetype = 'square'
diameter = 0.20
torqueTubeMaterial = 'black'       # IT's NOT GRAY, IT's GREY
cellLevelModule = False
numcellsx=0
numcellsy=0

# SceneDict Parameters
pitch = 4
albedo = 0.62 
clearance_height = 0.5  
nMods = 20
nRows = 7
azimuth_ang=180
tilt =30

In [6]:
# MAIN LOOP: Without and WITH TorqueTube
for foo in range (0, 2): 
    if foo==0:
        simulationname='No_torquetube'
        moduletype=simulationname
        torqueTube = False

    if foo==1:
        simulationname='With_torquetube'
        moduletype=simulationname
        torqueTube = True

    # Generate writefiletitle automatically for both cases
    writefiletitle=os.path.join(testfolder,'results', simulationname+'_'+str(timeindexstart)+'_'+str(timeindexend)+'.csv')

    print ("***Writing results to: %s \n\n" % writefiletitle)
    
    demo = RadianceObj(simulationname,path = testfolder)  
    demo.setGround(albedo) 
    epwfile = demo.getEPW(37.5,-77.6) 
    metdata = demo.readEPW(epwfile) 

    # Making module with all the variables
    moduleDict=demo.makeModule(name=moduletype,x=x,y=y,bifi=1, 
               torquetube=torqueTube, diameter = diameter, tubetype = tubetype, 
                               material = torqueTubeMaterial, zgap = zgap, numpanels = numpanels, ygap = ygap, 
                               rewriteModulefile = True, xgap=xgap, 
                               axisofrotationTorqueTube=axisofrotationTorqueTube)
    # create a scene with all the variables
    sceneDict = {'tilt':tilt,'pitch': pitch,'clearance_height':clearance_height,'azimuth':azimuth_ang, 'module_type':simulationname, 'nMods': nMods, 'nRows': nRows}  
    scene = demo.makeScene(moduletype=moduletype, sceneDict=sceneDict)

    # Now lot of stuff to format the CSV with the right headers. 
    with open (writefiletitle,'w') as csvfile:

        sw = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL, lineterminator='\n')

        outputheader=['Latitude(deg)','Longitude(deg)', 'Tilt(deg)']
        outputheadervars=[metdata.latitude, metdata.longitude, sceneDict['tilt']]

        if axisofrotationTorqueTube==False:
            outputheader+=['axisofrotationTorqueTube']
            outputheadervars.append(axisofrotationTorqueTube)

        sw.writerow(outputheader)
        sw.writerow(outputheadervars)

        outputheadertitles=['Timestamp','Back Avg', 'FrontAvg', 'Ratio']

        xheader=[]
        yheader=[]
        zheader=[]
        zrearheader=[]
        mattypeheader=[]
        rearmattypeheader=[]
        frontIrradheader=[]
        backIrradheader=[]
        import string

        for i in range (0, sensorsy):
            xheader.append('x_'+str(i))
            yheader.append('y_'+str(i))
            zheader.append('z_'+str(i))
            zrearheader.append('rearZ_'+str(i))
            mattypeheader.append('mattype_'+str(i))
            rearmattypeheader.append('rearMat_'+str(i))
            frontIrradheader.append('Wm2Front_'+str(i))
            backIrradheader.append('Wm2Back_'+str(i))

        outputheadertitles+=xheader
        outputheadertitles+=yheader
        outputheadertitles+=zheader
        outputheadertitles+=zrearheader
        outputheadertitles+=mattypeheader
        outputheadertitles+=rearmattypeheader
        outputheadertitles+=frontIrradheader
        outputheadertitles+=backIrradheader


        sw.writerow(outputheadertitles)

        for timeindex in range (timeindexstart,timeindexend):  #2160
            demo.gendaylit(metdata,timeindex) 
            octfile = demo.makeOct(demo.getfilelist())  
            analysis = AnalysisObj(octfile, demo.name)  
            
            #hubheight= (sceneDict['height']+moduleDict['sceney']/2.0*np.sin(sceneDict['tilt']*np.pi/180))

            frontscan, backscan = analysis.moduleAnalysis(scene, sensorsy=sensorsy)

            frontDict, backDict=analysis.analysis(octfile, simulationname+str(timeindex), frontscan, backscan)  # compare the back vs front irradiance
            
            if frontDict is not None:
                print('Annual bifacial ratio average:  %0.3f \n' %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) ) )
                outputvalues=[]
                outputvalues.append(timeindex)
                outputvalues.append(sum(analysis.Wm2Back))
                outputvalues.append(sum(analysis.Wm2Front))
                outputvalues.append( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) )

                # dictionary
                dictvalues=[]
                dictvalues.append(frontDict['x'])
                dictvalues.append(frontDict['y'])
                dictvalues.append(frontDict['z'])
                dictvalues.append(backDict['z'])
                dictvalues.append(frontDict['mattype'])
                dictvalues.append(backDict['mattype'])
                dictvalues.append(frontDict['Wm2'])
                dictvalues.append(backDict['Wm2'])
                flattened = [val for sublist in dictvalues for val in sublist]
                outputvalues+=flattened
                outputclean = str(outputvalues).translate(string.maketrans('', ''), '[]\'')
                sw.writerow(outputvalues)

    print("\n\n Finished Part %s ******************** \n ****************\n\n" % foo)
    
print ("Finished finished")


***Writing results to: C:\Users\sayala\Documents\RadianceScenes\Demo4\results\No_torquetube_2150_2152.csv 


path = C:\Users\sayala\Documents\RadianceScenes\Demo4
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw
 ... OK!


c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\main.py:2449: pvlibDeprecationWarning: The get_sun_rise_set_transit function was deprecated in pvlib 0.6.1 and will be removed in 0.7. Use sun_rise_set_transit_spa instead.
  sunup= pvlib.irradiance.solarposition.get_sun_rise_set_transit(datetimetz, lat, lon) #only for pvlib <0.6.1



Module Name: No_torquetube
REWRITING pre-existing module file. 
Module No_torquetube successfully created
Created No_torquetube.oct
Linescan in process: No_torquetube2150_Front
Linescan in process: No_torquetube2150_Back
Saved: results\irr_No_torquetube2150.csv
Annual bifacial ratio average:  0.300 



AttributeError: module 'string' has no attribute 'maketrans'

## B) Tracking Mismatch

Using same variables as fixed = except now the height is hub_height and the system is tracking.

### <u> Still under development </u> it's not currently saving all variables into the excel it should.

In [94]:
# Simulation Control Parameters
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Demo5'
timeindex = 4020 # Noon, June 17th. 
simulationname = 'Tracking_byHour_Example'
timeindexstart=2150
timeindexend=2152

hubheight=2.25

demo2 = RadianceObj(simulationname,path = testfolder) 
demo2.setGround(albedo) 
epwfile = demo2.getEPW(37.5,-77.6) 
metdata = demo2.readEPW(epwfile) 
trackerdict2 = demo2.set1axis(cumulativesky = False)  # this cumulativesky = False key is crucial to set up the hourly workflow
trackerdict2 = demo2.gendaylit1axis()  # optional parameters 'startdate', 'enddate' inputs = string 'MM/DD' or 'MM_DD' 

# MAIN LOOP: Without and WITH TorqueTube
for foo in range (0, 2): 
    if foo==0:
        simulationname='No_torquetube'
        moduletype=simulationname
        torqueTube = False

    if foo==1:
        simulationname='With_torquetube'
        moduletype=simulationname
        torqueTube = True

    # Generate writefiletitle automatically for both cases
    writefiletitle=os.path.join(testfolder,'results', simulationname+'_'+str(timeindexstart)+'_'+str(timeindexend)+'.csv')

    print ("***Writing results to: %s \n\n" % writefiletitle)
    

    # Making module with all the variables
    moduleDict=demo2.makeModule(name=moduletype,x=x,y=y,bifi=1, 
               torquetube=torqueTube, diameter = diameter, tubetype = tubetype, 
                               material = torqueTubeMaterial, zgap = zgap, numpanels = numpanels, ygap = ygap, 
                               rewriteModulefile = True, xgap=xgap, 
                               axisofrotationTorqueTube=axisofrotationTorqueTube, cellLevelModule=cellLevelModule, 
                               numcellsx=numcellsx, numcellsy = numcellsy)
    # create a scene with all the variables
    # SINCE THIS ISTRACKING, Do not need tilt. Most of the procedure is done fist (creating all skyfiles and rad files)
    sceneDict = {'pitch': pitch,'hub_height':hubheight, 'nMods': nMods, 'nRows': nRows}  
    trackerdict2 = demo2.makeScene1axis(trackerdict2, moduletype,sceneDict, cumulativesky = False) #makeScene creates a .rad file with 20 modules per row, 7 rows.

    # Now lot of stuff to format the CSV with the right headers. 
    with open (writefiletitle,'w') as csvfile:

        sw = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL, lineterminator='\n')

        outputheader=['Latitude(deg)','Longitude(deg)']
        outputheadervars=[metdata.latitude, metdata.longitude]

        if axisofrotationTorqueTube==False:
            outputheader+=['axisofrotationTorqueTube']
            outputheadervars.append(axisofrotationTorqueTube)

        sw.writerow(outputheader)
        sw.writerow(outputheadervars)

        outputheadertitles=['Timestamp','Back Avg', 'FrontAvg', 'Ratio']

        xheader=[]
        yheader=[]
        zheader=[]
        zrearheader=[]
        mattypeheader=[]
        rearmattypeheader=[]
        frontIrradheader=[]
        backIrradheader=[]
        import string

        for i in range (0, sensorsy):
            xheader.append('x_'+str(i))
            yheader.append('y_'+str(i))
            zheader.append('z_'+str(i))
            zrearheader.append('rearZ_'+str(i))
            mattypeheader.append('mattype_'+str(i))
            rearmattypeheader.append('rearMat_'+str(i))
            frontIrradheader.append('Wm2Front_'+str(i))
            backIrradheader.append('Wm2Back_'+str(i))

        outputheadertitles+=xheader
        outputheadertitles+=yheader
        outputheadertitles+=zheader
        outputheadertitles+=zrearheader
        outputheadertitles+=mattypeheader
        outputheadertitles+=rearmattypeheader
        outputheadertitles+=frontIrradheader
        outputheadertitles+=backIrradheader


        sw.writerow(outputheadertitles)

        # Now to make the specific Octs and the specific Analysis for each timestamp desired
        for timeindex in range (timeindexstart,timeindexend):  
            
            #converting timestamp to format needed by 1axis routine
            footime=metdata.datetime[timeindex]
            time = str(footime)[5:-12].replace('-','_').replace(' ','_')

            trackerdict2 = demo2.makeOct1axis(trackerdict2,singleindex=time)
            trackerdict2 = demo2.analysis1axis(trackerdict2,singleindex=time, customname=simulationname, modWanted=4, rowWanted=2)
                        
            if 'Wm2Front' in trackerdict2[time]:  # Check if analysis returned values correctly, and if so save in excel.    
                analysis = trackerdict2[time]['AnalysisObj']

                print('Annual bifacial ratio average:  %0.3f \n' %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) ) )
                outputvalues=[]
                outputvalues.append(timeindex)
                outputvalues.append(sum(analysis.Wm2Back))
                outputvalues.append(sum(analysis.Wm2Front))
                outputvalues.append( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) )
                
                # dictionary
                dictvalues=[]
                
                dictvalues.append(analysis.x)
                dictvalues.append(analysis.y)
                dictvalues.append(analysis.z)
                dictvalues.append(analysis.rearZ)
                dictvalues.append(analysis.mattype)
                dictvalues.append(analysis.rearMat)
                dictvalues.append(analysis.Wm2Front)
                dictvalues.append(analysis.Wm2Back)
                flattened = [val for sublist in dictvalues for val in sublist]
                outputvalues+=flattened
                outputclean = str(outputvalues).translate(string.maketrans('', ''), '[]\'')
                sw.writerow(outputvalues)

    print("\n\n Finished Part %s ******************** \n ****************\n\n" % foo)

print ("Finished finished")


***Writing results to: C:\Users\sayala\Documents\RadianceScenes\Demo5\results\No_torquetube_2150_2152.csv 


path = C:\Users\sayala\Documents\RadianceScenes\Demo5
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!
('\nModule Name:', 'No_torquetube')
Module file did not exist before, creating new module file
Module No_torquetube successfully created
Creating ~4000 skyfiles.  Takes 1-2 minutes
Created 4320 skyfiles in /skies/

Making ~4000 .rad files for gendaylit 1-axis workflow (this takes a minute..)
4320 Radfiles created in /objects/

Making 1 octfiles for 1-axis tracking in root directory.
Created 1axis_03_31_15.oct
Linescan in process: 1axis_03_31_15No_torquetube_Front
Linescan in process: 1axis_03_31_15No_torquetube_Back
Saved: results\irr_1axis_03_31_15No_torquetube.csv
Index: 03_31_15. Wm2Front: 135.505688889. Wm2Back: 60.5064466667
Annual bifacial ratio average:  0.447 


Making 1 octfiles for 1-axis tracking in root directory.
Created 1axis_03_31_16.oct
Line

In [80]:
# ANALYSIS OBJECT CONTENT SANITY CHEcK:
analysis = trackerdict2[time]['AnalysisObj']
sum(analysis.Wm2Back) # works
analysis.rearMat # Work s
analysis.rearZ # Works
# analysis.y ## NOT WORKING
# how to access line 2531 " data_sub = {key:data[key] for key in ['x', 'y', 'z', 'Wm2', 'mattype']}"data on AnalysisObj?

AttributeError: AnalysisObj instance has no attribute 'y'